In [1]:
from benchmark import impl
from benchmark import alphaxutils
from decimal import Decimal
import pandas as pd

Order = alphaxutils.Order
tradeBySourceAmount = alphaxutils.tradeBySourceAmount
tradeByTargetAmount = alphaxutils.tradeByTargetAmount
AlphaRouter = alphaxutils.AlphaRouter
CarbonOrderUI = alphaxutils.CarbonOrderUI
assertAlmostEqual = alphaxutils.assertAlmostEqual

In [2]:
threshold_orders = 6
inputAmount = Decimal('3')
support_partial = True

order_params = {
    'liquidity':Decimal('5'),
    'highestRate':Decimal('0.0005'), # 2000
    'lowestRate':Decimal('0.0004'),  # 2500
    'marginalRate':Decimal('0.0005'),
    }
order_params2 = {
    'liquidity':Decimal('4'),
    'highestRate':Decimal('0.0006'), # 1666
    'lowestRate':Decimal('0.0005'),  # 2000
    'marginalRate':Decimal('0.0006'),
    }

In [3]:
orders = []
for i in range(5):
    orders += [Order(order_params)]
    orders += [Order(order_params2)]

indexes = list(range(len(orders)))   

# Match by Target

In [4]:
hypothetical_output_amts = {i: tradeByTargetAmount(x=inputAmount, order=orders[i])[0] for i in indexes}
actual_output_amts = {i: tradeByTargetAmount(x=orders[i].y, order=orders[i])[0] for i in indexes}
ordered_associated_liquidity = {i:orders[i].y for i in actual_output_amts.keys()}

results = pd.DataFrame(
    [
        hypothetical_output_amts.keys(),
        hypothetical_output_amts.values(),
        actual_output_amts.keys(),
        actual_output_amts.values(),
    ],
    index=[
        "indexes",
        "hypothetical_output_amts",
        "indexes_b",
        "actual_output_amts",
    ],
)
results = results.T.copy()
assert(list(results.indexes) == list(results.indexes_b))

results.sort_values(
    by=["actual_output_amts", "hypothetical_output_amts", 'indexes'], ascending=[True, True, True], inplace=True
)

results2 = pd.DataFrame(
    [
        ordered_associated_liquidity.keys(),
        ordered_associated_liquidity.values(),
    ],
    index=[
        "ordered_associated_liquidity_keys",
        "ordered_associated_liquidity",
    ],
)
results2 = results2.T.copy()

results = pd.merge(results, results2, how='left', left_on = 'indexes', right_on='ordered_associated_liquidity_keys')

results.fillna(0, inplace=True)
results.reset_index(inplace=True, drop=True)
results

,indexes,hypothetical_output_amts,indexes_b,actual_output_amts,ordered_associated_liquidity_keys,ordered_associated_liquidity
0,1,5349.57780777178636087645819693541744542684146...,1,7302.96743340221484609293043734402845270326259...,1,4
1,3,5349.57780777178636087645819693541744542684146...,3,7302.96743340221484609293043734402845270326259...,3,4
2,5,5349.57780777178636087645819693541744542684146...,5,7302.96743340221484609293043734402845270326259...,5,4
3,7,5349.57780777178636087645819693541744542684146...,7,7302.96743340221484609293043734402845270326259...,7,4
4,9,5349.57780777178636087645819693541744542684146...,9,7302.96743340221484609293043734402845270326259...,9,4
5,0,6405.76474687263408460320377322685764870695654...,0,11180.3398874989484820458683436563811772030917...,0,5
6,2,6405.76474687263408460320377322685764870695654...,2,11180.3398874989484820458683436563811772030917...,2,5
7,4,6405.76474687263408460320377322685764870695654...,4,11180.3398874989484820458683436563811772030917...,4,5
8,6,6405.76474687263408460320377322685764870695654...,6,11180.3398874989484820458683436563811772030917...,6,5
9,8,6405.76474687263408460320377322685764870695654...,8,11180.3398874989484820458683436563811772030917...,8,5


## Partial Match Logic

In [5]:
if (not support_partial) & (results.ordered_associated_liquidity.sum() < abs(inputAmount)):
    print('Insufficient Liquidity')
else:
    passed_indexes = AlphaRouter.gen_one_order_selector(results.ordered_associated_liquidity, abs(inputAmount), threshold_orders)
    top_n_threshold_orders = [results.indexes[i] for i in passed_indexes]
order_subset = [orders[i] for i in top_n_threshold_orders]
top_n_threshold_orders

[1, 3, 5, 7, 9, 0]

## Exact Matching with Marginal Price Router

In [6]:
# def mpr_match(inputAmount, order_subset):
total_subset_liquidity = sum(o.y for o in order_subset)
print('inputAmount', inputAmount)
print('total_subset_liquidity', total_subset_liquidity)
if inputAmount>= total_subset_liquidity:
    print(f'Partial Match - using {total_subset_liquidity}')
    inputAmount = total_subset_liquidity
    rl1 = [o.y for o in order_subset]
    rl2 = [o.dxfromdy_f(o.y) for o in order_subset]
else:
    dy_f = lambda p: sum(o.dyfromp_f(p) for o in order_subset)
    dx_f = lambda p: sum(o.dxfromdy_f(o.dyfromp_f(p)) for o in order_subset)
    p_goal = CarbonOrderUI.goalseek(lambda p: dy_f(p)-inputAmount, Decimal('0.000000001'), Decimal('1000000000'))
    rl1 = [o.dyfromp_f(p_goal) for o in order_subset]
    rl2 = [o.dxfromdy_f(o.dyfromp_f(p_goal)) for o in order_subset]

actions = {top_n_threshold_orders[i]:{"input":rl1[i],"output":rl2[i]} for i in range(len(top_n_threshold_orders))}
assertAlmostEqual(inputAmount, sum(rl1), Decimal('1E-8'))
print('total_input',sum(rl1))
print('total_output', sum(rl2))
print('effective_price', sum(rl2) / sum(rl1))
# return(actions)
actions

inputAmount 3
total_subset_liquidity 25
total_input 3.0000000004636469297871570538814313141381862659480
total_output 0.0017764751511482414262144124182395593868113886272374
effective_price 0.00059215838362456300323305274804614533153029350471392


{1: {'input': Decimal('0.6000000000927293859574314107762862628276372531896'),
  'output': Decimal('0.00035529503022964828524288248364791187736227772544747')},
 3: {'input': Decimal('0.6000000000927293859574314107762862628276372531896'),
  'output': Decimal('0.00035529503022964828524288248364791187736227772544747')},
 5: {'input': Decimal('0.6000000000927293859574314107762862628276372531896'),
  'output': Decimal('0.00035529503022964828524288248364791187736227772544747')},
 7: {'input': Decimal('0.6000000000927293859574314107762862628276372531896'),
  'output': Decimal('0.00035529503022964828524288248364791187736227772544747')},
 9: {'input': Decimal('0.6000000000927293859574314107762862628276372531896'),
  'output': Decimal('0.00035529503022964828524288248364791187736227772544747')},
 0: {'input': 0, 'output': Decimal('0E+46')}}

In [7]:
dy_f = lambda p: sum(o.dyfromp_f(p) for o in order_subset)
dx_f = lambda p: sum(o.dxfromdy_f(o.dyfromp_f(p)) for o in order_subset)
p_goal = CarbonOrderUI.goalseek(lambda p: dy_f(p)-inputAmount, Decimal('0.000000001'), Decimal('1000000000'))
rl1 = [o.dyfromp_f(p_goal) for o in order_subset]
rl2 = [o.xfromy_f(o.dyfromp_f(p_goal)) for o in order_subset]  # dxfromdy_f
rl1

[Decimal('0.6000000000927293859574314107762862628276372531896'),
 Decimal('0.6000000000927293859574314107762862628276372531896'),
 Decimal('0.6000000000927293859574314107762862628276372531896'),
 Decimal('0.6000000000927293859574314107762862628276372531896'),
 Decimal('0.6000000000927293859574314107762862628276372531896'),
 0]

In [8]:
rl2

[Decimal('6119.9050618371375475892898458954906013785696571637'),
 Decimal('6119.9050618371375475892898458954906013785696571637'),
 Decimal('6119.9050618371375475892898458954906013785696571637'),
 Decimal('6119.9050618371375475892898458954906013785696571637'),
 Decimal('6119.9050618371375475892898458954906013785696571637'),
 Decimal('11180.339887498948482045868343656381177203091798058')]

In [10]:
[rl1[i]/rl2[i] for i in range(len(rl2))]

[Decimal('0.000098040736584991249453113741471322418101098178487865'),
 Decimal('0.000098040736584991249453113741471322418101098178487865'),
 Decimal('0.000098040736584991249453113741471322418101098178487865'),
 Decimal('0.000098040736584991249453113741471322418101098178487865'),
 Decimal('0.000098040736584991249453113741471322418101098178487865'),
 Decimal('0E+45')]

In [9]:
dx_f = lambda p: sum(o.dxfromdy_f(o.dyfromp_f(p)) for o in order_subset)
p_goal = CarbonOrderUI.goalseek(lambda p: dx_f(p)-inputAmount, Decimal('0.000000001'), Decimal('1000000000'))
p_goal

ValueError: ('Sign of f(a) must be opposite of sign of f(b)', Decimal('-2.9868094808723968880344514306752526810855044877404'), Decimal('-3'), Decimal('1E-9'), Decimal('1000000000'))

In [ ]:
1/((0.0005 * 0.0006)**.5) * 4

In [ ]:
1/((0.0005 * 0.0004)**.5) * 5

In [ ]:
# mpr_match(inputAmount, order_subset)